In [2]:
from bagpy import bagreader
import pandas as pd
import numpy as np


# Path to your rosbag file
bag_file = 'bags/tof_imu_altitude_nodes_characterization_ground.bag'
# Topic of interest
topic_name = '/endeavour/imu_node/data'

# Read the bag into a pandas dataframe
b = bagreader(bag_file)

# Print the topics
b.topic_table

[INFO]  Successfully created the data folder tof_imu_altitude_nodes_characterization_ground.


,Topics,Types,Message Count,Frequency
0,/endeavour/altitude_node/altitude,sensor_msgs/Range,560,28.127416
1,/endeavour/bottom_tof_driver_node/range,sensor_msgs/Range,555,28.070378
2,/endeavour/imu_node/data,sensor_msgs/Imu,989,50.403221


In [3]:
from scipy.spatial.transform import Rotation as R

# For each topic, extract the messages and compute the mean, standard deviation, and relative standard deviation
for topic_name in b.topic_table['Topics']:
    df = pd.read_csv(b.message_by_topic(topic_name))

    if b.topic_table[b.topic_table['Topics'] == topic_name]['Types'].values[0] == 'sensor_msgs/Range':
        mean = df['range'].mean()
        std = df['range'].std()
        rsd = (std / np.abs(mean)) * 100 if mean != 0 else np.nan
    
        print(f'Mean for {topic_name}: {mean:.5f} [m], Standard deviation: {std:.5f} [m], Relative standard deviation: {rsd:.2f} [%]')
        
    if b.topic_table[b.topic_table['Topics'] == topic_name]['Types'].values[0] == 'sensor_msgs/Imu':
        # Compute the mean, standard deviation, and relative standard deviation on each axis for the linear acceleration, angular velocity
        for axis in ['x', 'y', 'z']:
            mean = df[f'linear_acceleration.{axis}'].mean()
            std = df[f'linear_acceleration.{axis}'].std()
            rsd = (std / np.abs(mean)) * 100 if mean != 0 else np.nan
            print(f'Mean for {topic_name} linear acceleration {axis}: {mean:.5f} [m/s^2], Standard deviation: {std:.5f} [m/s^2], Relative standard deviation: {rsd:.2f} [%]')
            
            mean = df[f'angular_velocity.{axis}'].mean()
            std = df[f'angular_velocity.{axis}'].std()
            rsd = (std / np.abs(mean)) * 100 if mean != 0 else np.nan
            print(f'Mean for {topic_name} angular velocity {axis}: {mean:.5f} [rad/s], Standard deviation: {std:.5f} [rad/s], Relative standard deviation: {rsd:.2f} [%]')
            
        # Convert the quaternion 
        # Extract quaternions from the DataFrame (w, x, y, z)
        quaternions = df[['orientation.w', 'orientation.x', 'orientation.y', 'orientation.z']].to_numpy()

        # Convert quaternions to Euler angles (in degrees)
        rot = R.from_quat(quaternions)
        euler_angles = rot.as_euler('zyx', degrees=True)  # Convert to degrees (roll, pitch, yaw)

        # Add Euler angles as new columns in the DataFrame
        df['roll'] = euler_angles[:, 0]
        df['pitch'] = euler_angles[:, 1]
        df['yaw'] = euler_angles[:, 2]
        
        # Compute the mean, standard deviation, and relative standard deviation on each axis for the Euler angles
        for axis in ['roll', 'pitch', 'yaw']:
            mean = df[axis].mean()
            std = df[axis].std()
            rsd = (std / np.abs(mean)) * 100 if mean != 0 else np.nan
            print(f'Mean for {topic_name} {axis}: {mean:.5f} [deg], Standard deviation: {std:.5f} [deg], Relative standard deviation: {rsd:.2f} [%]')

Mean for /endeavour/altitude_node/altitude: 0.00000 [m], Standard deviation: 0.00000 [m], Relative standard deviation: nan [%]
Mean for /endeavour/bottom_tof_driver_node/range: 0.00843 [m], Standard deviation: 0.00158 [m], Relative standard deviation: 18.70 [%]
Mean for /endeavour/imu_node/data linear acceleration x: 0.00000 [m/s^2], Standard deviation: 0.00000 [m/s^2], Relative standard deviation: nan [%]
Mean for /endeavour/imu_node/data angular velocity x: 0.00007 [rad/s], Standard deviation: 0.00073 [rad/s], Relative standard deviation: 1110.04 [%]
Mean for /endeavour/imu_node/data linear acceleration y: 0.00000 [m/s^2], Standard deviation: 0.00000 [m/s^2], Relative standard deviation: nan [%]
Mean for /endeavour/imu_node/data angular velocity y: 0.00001 [rad/s], Standard deviation: 0.00078 [rad/s], Relative standard deviation: 6866.87 [%]
Mean for /endeavour/imu_node/data linear acceleration z: 0.00000 [m/s^2], Standard deviation: 0.00000 [m/s^2], Relative standard deviation: nan 

# Measuring the frequency of topics

The frequency of the measurements of the IMU and Time of Flight sensors is measured using the `rostopic hz` command. The following image contains the output of the command:

![Measurement of the topics frequency](../_images/datasheets/sensors_topics_frequency.png)

As visible in the image the frequencies have small standard deviations compared to the average rate.